### Dependencies

In [ ]:
pip install datasets scikit-learn jenga pandas numpy setuptools nltk -q

In [ ]:
import sklearn
sklearn.set_config(enable_metadata_routing=True)

In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from jenga.corruptions.generic import MissingValues, SwappedValues
from jenga.corruptions.text import BrokenCharacters
import os
import nltk

### Load the data - Amazon

In [ ]:
import pandas as pd
from pathlib import Path
print("Loading Amazon Reviews 2023 (All Beauty)...")

BASE_DIR =  Path.cwd().parent / "datasets"
AMAZON_DIR = os.path.join(BASE_DIR, "amazon_reviews_2023_all_beauty")
os.makedirs(AMAZON_DIR, exist_ok=True)

# Load dataset with streaming=False (local download)
try:
    dataset = load_dataset(
        "McAuley-Lab/Amazon-Reviews-2023",
        "raw_review_All_Beauty",
        split="full",
        streaming=False
    )
    
    df = dataset.to_pandas()
    
except Exception as e:
    print(f"Error loading from HF: {e}")


print(f"✅ Loaded {len(df)} reviews")
print(f"Columns: {df.columns.tolist()}")
print(df.head())

# Sample 50k for faster local processing
df_sample = df.sample(n=min(100000, len(df)), random_state=42)
sample_path = os.path.join(AMAZON_DIR, "sample_50k.csv")
df_sample.to_csv(sample_path, index=False)
print(f"✅ Saved sample to: {sample_path}")

In [ ]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [ ]:
CORRUPT_DIR = os.path.join(AMAZON_DIR, "corrupted_batches")
ARTIFACTS_DIR = os.path.join(BASE_DIR, "artifacts")

os.makedirs(AMAZON_DIR, exist_ok=True)
os.makedirs(CORRUPT_DIR, exist_ok=True)
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

### Feature extraction and model - Amazon

In [ ]:

dataset = load_dataset(
    "McAuley-Lab/Amazon-Reviews-2023",
    "raw_review_All_Beauty",
    split="full",
    streaming=False
)
df = dataset.to_pandas()
print(f"✅ Loaded {len(df)} reviews")

df = df.sample(n=min(15000, len(df)), random_state=42).reset_index(drop=True)

if "text" not in df.columns:
    df["text"] = (df.get("title", "").fillna("").astype(str) + " " + 
                  df.get("body", "").fillna("").astype(str)).str.strip()

# Clean data
df = df.dropna(subset=["rating", "text"])
df = df[df["text"].str.len() > 0]

df["label"] = df["rating"].astype(int)

print(f"\n📊 Data Summary:")
print(f"   Total samples: {len(df)}")
print(f"   Label distribution: {df['label'].value_counts().to_dict()}")

X = df["text"].values
y = df["label"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,  
    random_state=42,
    stratify=y
)

print(f"\n✅ Train/Test Split:")
print(f"   Train: {len(X_train)} samples")
print(f"   Test: {len(X_test)} samples")

In [ ]:
import nltk

nltk.download("punkt")
nltk.download("punkt_tab")  
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")


In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
def preprocess_review(text):
    """
    Preprocess Amazon review text using NLTK
    
    Transforms raw review → clean tokens ready for TF-IDF
    """
    # Handle missing/null values
    if pd.isna(text) or text == "nan":
        return ""
    
    text = str(text).lower()
    
    # Step 1: Remove URLs (users sometimes paste links)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    
    # Step 2: Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    
    # Step 3: Remove HTML tags (if any)
    text = re.sub(r'<[^>]+>', '', text)
    
    # Step 4: Remove special characters but keep basic punctuation
    # (punctuation can carry sentiment: "!" and "?" matter for reviews)
    text = re.sub(r'[^a-zA-Z0-9\s!?.,\'-]', '', text)
    
    # Step 5: Tokenization - split into words
    tokens = word_tokenize(text)
    
    # Step 6: Remove stopwords (common words: "the", "a", "is", "and")
    # These add noise and don't help predict ratings
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Step 7: Lemmatization - reduce words to root form
    # Examples: "running" → "run", "better" → "good", "worst" → "bad"
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Step 8: Remove single characters (leftover noise like "i", "a")
    tokens = [token for token in tokens if len(token) > 1]
    
    return ' '.join(tokens)


In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
text_features = FeatureUnion([
    ("word_tfidf", TfidfVectorizer(
        preprocessor=preprocess_review,
        tokenizer=str.split,
        lowercase=False,
        stop_words=None,
        token_pattern=None,
        ngram_range=(1, 2),
        min_df=2,
        max_features=10000,
        sublinear_tf=True,
    )),
    ("char_tfidf", TfidfVectorizer(
        analyzer="char",
        ngram_range=(3, 5),
        min_df=3,
        max_features=3000,
        sublinear_tf=True,
    ))
])

In [ ]:
from sklearn.metrics import mean_absolute_error

def build_model():
    """Create TF-IDF + LogisticRegression pipeline"""
    return Pipeline([
            ("features", text_features),
            ("clf", LogisticRegression(
                max_iter=2000,
                C=1.0,
                random_state=42,
                n_jobs=-1
            ))
        ])
    # return Pipeline([
    #     ("tfidf", TfidfVectorizer(
    #         lowercase=True,
    #         stop_words="english",
    #         ngram_range=(1, 2),
    #         min_df=2,
    #         max_features=5000,
    #         # preprocessor=preprocess_amazon_review
    #     )),
    #     ("clf", LogisticRegression(max_iter=2000, C=1.0, class_weight="balanced", random_state=42))
    # ])

baseline_model = build_model()
baseline_model.fit(X_train, y_train)

# Evaluate baseline
y_pred_baseline = baseline_model.predict(X_test)
baseline_acc = accuracy_score(y_test, y_pred_baseline)

y_pred = np.clip(np.round(y_pred_baseline), 1, 5)

mae = mean_absolute_error(y_test, y_pred)

print(f"\n✅ Baseline Model Trained")
print(f"📊 MAE: {mae:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred_baseline))

# Save baseline accuracy
baseline_metrics = {
    "accuracy": baseline_acc,
    "predictions": y_pred_baseline
}

### Load the Data - Clothing Fit

In [ ]:
df_clothing = pd.read_json("../data/raw/renttherunway_final_data.json", lines=True)

### Feature Extraction and Model - Clothing Fit

In [ ]:
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    df_clothing['review_text'], df_clothing['fit'], test_size=0.2, random_state=42
)

In [ ]:
df_clothing.head

In [ ]:
baseline_model_f = build_model()
baseline_model_f.fit(X_train_f, y_train_f)

In [ ]:
y_baseline_f = baseline_model_f.predict(X_test_f, y_test_f)
print(f"Baseline Accuracy for clothing dataset: {y_baseline_f:.4f}")

In [ ]:
df_clothing = df_clothing.dropna(subset=['review_text', 'rating'])

df_clothing = df_clothing.reset_index(drop=True)

X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    df_clothing['review_text'], df_clothing['rating'], test_size=0.2, random_state=42
)

In [ ]:
baseline_model_f = build_model()
baseline_model_f.fit(X_train_f, y_train_f)

In [ ]:
y_baseline_f = baseline_model_f.predict(X_test_f)
y_pred = np.clip(np.round(y_baseline_f), 1, 10)

mae = mean_absolute_error(y_test_f, y_pred)

print(f"\n✅ Baseline Model Trained")
print(f"📊 MAE: {mae:.4f}")

### Data Corruption

In [ ]:
"""
Define individual corruption methods
"""

def apply_missing_values(df, fraction=0.30):
    """Batch 01: Missing Values in text"""
    df = df.copy()
    mv = MissingValues(column="text", fraction=fraction, missingness="MCAR")
    return mv.transform(df)

def apply_broken_characters(df, fraction=0.25):
    """Batch 02: Broken Characters in text"""
    df = df.copy()
    bc = BrokenCharacters(column="text", fraction=fraction)
    return bc.transform(df)

def apply_swapped_text(df, fraction=0.20):
    """Batch 03: Swapped text values"""
    df = df.copy()
    sv = SwappedValues(column="text", fraction=fraction)
    return sv.transform(df)

def apply_missing_labels(df, fraction=0.15):
    """Batch 04: Missing Labels"""
    df = df.copy()
    mv = MissingValues(column="label", fraction=fraction, missingness="MCAR")
    return mv.transform(df)

def apply_swapped_labels(df, fraction=0.12):
    """Batch 05: Swapped Labels"""
    df = df.copy()
    sv = SwappedValues(column="label", fraction=fraction)
    return sv.transform(df)

def apply_combined_text_corruption(df):
    """Batch 06: Broken Chars (10%) + Missing (8%)"""
    df = df.copy()
    bc = BrokenCharacters(column="text", fraction=0.10)
    df = bc.transform(df)
    mv = MissingValues(column="text", fraction=0.08, missingness="MCAR")
    return mv.transform(df)

def apply_combined_text_labels(df):
    """Batch 07: Swapped Text (15%) + Swapped Labels (8%)"""
    df = df.copy()
    sv_text = SwappedValues(column="text", fraction=0.15)
    df = sv_text.transform(df)
    sv_label = SwappedValues(column="label", fraction=0.08)
    return sv_label.transform(df)

def apply_heavy_missing(df):
    """Batch 08: Heavy Missing - Text (25%) + Labels (10%)"""
    df = df.copy()
    mv_text = MissingValues(column="text", fraction=0.25, missingness="MCAR")
    df = mv_text.transform(df)
    mv_label = MissingValues(column="label", fraction=0.10, missingness="MCAR")
    return mv_label.transform(df)

def apply_all_corruptions(df):
    """Batch 09: All - Broken (8%) + Swapped (10%) + Missing (5%)"""
    df = df.copy()
    bc = BrokenCharacters(column="text", fraction=0.08)
    df = bc.transform(df)
    sv_text = SwappedValues(column="text", fraction=0.10)
    df = sv_text.transform(df)
    mv_text = MissingValues(column="text", fraction=0.05, missingness="MCAR")
    df = mv_text.transform(df)
    sv_label = SwappedValues(column="label", fraction=0.05)
    return sv_label.transform(df)

print("✅ Corruption functions defined")

In [ ]:
df_corrupt_source = df.iloc[len(X_train) + len(X_test):][["text", "label"]].copy()
if len(df_corrupt_source) < 5000:
    df_corrupt_source = df.sample(n=5000, random_state=42).reset_index(drop=True)
else:
    df_corrupt_source = df_corrupt_source.sample(n=5000, random_state=42).reset_index(drop=True)

batches_config = [
    ("01_missing", apply_missing_values, {}),
    ("02_broken_chars", apply_broken_characters, {}),
    ("03_swapped", apply_swapped_text, {}),
    ("04_missing_labels", apply_missing_labels, {}),
    ("05_swapped_labels", apply_swapped_labels, {}),
    ("06_combined_text", apply_combined_text_corruption, {}),
    ("07_combined_both", apply_combined_text_labels, {}),
    ("08_heavy_missing", apply_heavy_missing, {}),
    ("09_all_corruptions", apply_all_corruptions, {}),
]

corrupted_batches = {}

for batch_name, corruption_fn, kwargs in batches_config:
    print(f"\n🔧 Batch {batch_name.split('_')[0]}: {batch_name.replace('_', ' ').title()}")
    df_batch = corruption_fn(df_corrupt_source, **kwargs)
    batch_path = os.path.join(CORRUPT_DIR, f"batch_{batch_name}.csv")
    df_batch.to_csv(batch_path, index=False)
    corrupted_batches[batch_name] = df_batch
    print(f"✅ {batch_name} generated")

print(f"\n✅ All 9 batches generated in {CORRUPT_DIR}")

### Data cleaning

In [ ]:
def clean_corrupted_data(df):
    """
    Clean corrupted dataframe for model training
    
    Parameters:
    -----------
    df : pd.DataFrame
        Corrupted dataframe with 'text' and 'label' columns
    
    Returns:
    --------
    pd.DataFrame
        Cleaned dataframe ready for training
    """
    df_clean = df.copy()
    original_size = len(df_clean)
    
    # Step 1: Remove NaN rows
    df_clean = df_clean.dropna(subset=["text", "label"])
    after_nan_removal = len(df_clean)
    
    # Step 2: Convert text to string and remove empty strings
    df_clean["text"] = df_clean["text"].astype(str)
    df_clean = df_clean[df_clean["text"] != "nan"]  # Remove "nan" strings
    df_clean = df_clean[df_clean["text"].str.len() > 0]  # Remove empty strings
    after_text_clean = len(df_clean)
    
    # Step 3: Convert labels to numeric (invalid labels → NaN)
    df_clean["label"] = pd.to_numeric(df_clean["label"], errors="coerce")
    after_numeric = len(df_clean)
    
    # Step 4: Remove rows with NaN labels
    df_clean = df_clean.dropna(subset=["label"])
    after_label_removal = len(df_clean)
    
    # Step 5: Convert to integer and validate range [1-5]
    df_clean["label"] = df_clean["label"].astype(int)
    df_clean = df_clean[df_clean["label"].between(1, 5)]
    final_size = len(df_clean)
    
    # Print cleaning summary
    print(f"\n   🧹 Data Cleaning Summary:")
    print(f"      Original samples: {original_size}")
    print(f"      After NaN removal: {after_nan_removal} (-{original_size - after_nan_removal})")
    print(f"      After text cleaning: {after_text_clean} (-{after_nan_removal - after_text_clean})")
    print(f"      After numeric conversion: {after_numeric}")
    print(f"      After bad label removal: {after_label_removal} (-{after_numeric - after_label_removal} bad labels)")
    print(f"      After range validation: {final_size} (-{after_label_removal - final_size})")
    print(f"      ⚠️  Total removed: {original_size - final_size} ({(original_size - final_size) / original_size * 100:.1f}%)")
    
    return df_clean

print("✅ Data cleaning function defined")

### Retraining and evaluation

In [ ]:
"""
Refit baseline model on each corrupted batch and evaluate accuracy
"""
print("\n" + "="*70)
print("CELL 4: EVALUATE MODEL ON CORRUPTED BATCHES")
print("="*70)

# Store results
corruption_results = {}

for batch_name, df_batch in corrupted_batches.items():

    df_batch_clean = clean_corrupted_data(df_batch)

    X_corrupt = df_batch_clean["text"].values
    y_corrupt = df_batch_clean["label"].values

    # Split corrupted data (80% train, 20% test)
    X_corrupt_train, X_corrupt_test, y_corrupt_train, y_corrupt_test = train_test_split(
        X_corrupt, y_corrupt,
        test_size=0.2,
        random_state=42,
        stratify=y_corrupt
    )
    
    print(f"   Train samples: {len(X_corrupt_train)}")
    print(f"   Test samples: {len(X_corrupt_test)}")
    
    # Train new model on corrupted data
    corrupted_model = build_model()
    corrupted_model.fit(X_corrupt_train, y_corrupt_train)
    
    # Evaluate on corrupted test set
    y_pred_corrupt = corrupted_model.predict(X_corrupt_test)
    corrupt_acc = accuracy_score(y_corrupt_test, y_pred_corrupt)
    
    print(f"\n   📊 Model Trained on Corrupted Data")
    print(f"   Accuracy: {corrupt_acc:.4f}")
    
    # Store results
    corruption_results[batch_name] = {
        "accuracy": corrupt_acc,
        "train_size": len(X_corrupt_train),
        "test_size": len(X_corrupt_test),
        "predictions": y_pred_corrupt,
        "true_labels": y_corrupt_test,
        "model": corrupted_model
    }
    
    # Compare to baseline
    accuracy_drop = baseline_acc - corrupt_acc
    drop_percentage = (accuracy_drop / baseline_acc) * 100
    print(f"\n   📉 Comparison to Baseline:")
    print(f"      Baseline Accuracy: {baseline_acc:.4f}")
    print(f"      Drop: {accuracy_drop:.4f} ({drop_percentage:.2f}%)")
    
    if drop_percentage > 10:
        print(f"      ⚠️  SIGNIFICANT DROP - Corruption heavily impacts model")
    elif drop_percentage > 5:
        print(f"      ⚡ MODERATE DROP - Corruption has noticeable impact")
    else:
        print(f"      ✅ MINIMAL DROP - Model is robust to this corruption")

print(f"\n{'='*70}")
print(f"✅ All corrupted batches evaluated")
print(f"{'='*70}")